In [54]:
from pinecone import Pinecone
from warnings import filterwarnings
from datasets import load_dataset
import os
pinecone = Pinecone(api_key = "")

os.environ["OPENAI_API_KEY"] = ""
filterwarnings("ignore")

In [3]:
pubmed = load_dataset("pubmed_qa", "pqa_labeled", split = "train")
pubmed

Dataset({
    features: ['pubid', 'question', 'context', 'long_answer', 'final_decision'],
    num_rows: 1000
})

In [5]:
pubmed[0]

{'pubid': 21645374,
 'question': 'Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?',
 'context': {'contexts': ['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.',
   'The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), ce

In [6]:
pubmed[0]["question"]

'Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?'

In [7]:
pubmed[0]["context"]

{'contexts': ['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants.',
  'The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A. madagascariensis. A single areole within a window stage leaf (PCD is occurring) was divided into three areas based on the progression of PCD; cells that will not undergo PCD (NPCD), cells in early stages of PCD (EPCD), and cells in late stages of PCD (LPCD). Window stage leaves were stained with the mitochondrial dye MitoT

In [20]:
limit = 384

def chunker(contexts: list):
    chunks = []
    all_contexts = ' '.join(contexts).split('.')
    chunk = []
    for context in all_contexts:
        chunk.append(context)
        if len(chunk) >= 3 and len('.'.join(chunk)) > limit:
            # surpassed limit so add to chunks and reset
            chunks.append('.'.join(chunk).strip()+'.')
            # add some overlap between passages
            chunk = chunk[-2:]
    # if we finish and still have a chunk, add it
    if chunk is not None:
        chunks.append('.'.join(chunk))
    return chunks
    

In [21]:
chunks = chunker(pubmed[0]["context"]["contexts"])
chunks

['Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature.',
 'The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants. The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A.',
 'The role of mitochondria during PCD has been recognized in animals; however, it has been less

In [29]:
data = []
for i in pubmed:
    chunks = chunker(i["context"]["contexts"])
    for x, context in enumerate(chunks):
        data.append({"id": f"{i['pubid']}-{x}", "context": context})
        
data[:10]

[{'id': '21645374-0',
  'context': 'Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature.'},
 {'id': '21645374-1',
  'context': 'The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature. The role of mitochondria during PCD has been recognized in animals; however, it has been less studied during PCD in plants. The following paper elucidates the role of mitochondrial dynamics during developmentally regulated PCD in vivo in A.'},
 {'id': '21645374-2',
  '

In [30]:
len(data)

5930

In [31]:
from sentence_transformers import SentenceTransformer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

dense_model = SentenceTransformer("msmarco-bert-base-dot-v5", device = device)

dense_model

config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<?, ?B/s] 
README.md: 100%|██████████| 6.19k/6.19k [00:00<?, ?B/s]
sentence_bert_config.json: 100%|██████████| 54.0/54.0 [00:00<?, ?B/s]
config.json: 100%|██████████| 636/636 [00:00<?, ?B/s] 
pytorch_model.bin: 100%|██████████| 438M/438M [00:44<00:00, 9.83MB/s] 
tokenizer_config.json: 100%|██████████| 461/461 [00:00<?, ?B/s] 
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 502kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 1.26MB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 190kB/s]


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
)

In [32]:
embeds = dense_model.encode(data[0]['context'])
embeds

array([-8.60981867e-02, -6.40461296e-02, -9.06743035e-02, -1.38834402e-01,
        4.03490871e-01,  4.51100431e-02,  1.78422630e-01,  8.63788836e-03,
        3.98673832e-01, -1.20012492e-01, -5.58831654e-02,  1.04059346e-01,
       -5.98424911e-01,  4.46074545e-01,  7.60736912e-02,  7.18574762e-01,
        1.38988554e-01, -3.24186347e-02,  5.96617907e-02,  5.81385940e-02,
       -1.46968052e-01,  2.05823295e-02,  7.17516482e-01,  2.62669086e-01,
        1.86890930e-01, -2.79622376e-01, -4.33417171e-01, -3.65013003e-01,
       -4.08249348e-01,  4.92232591e-01, -4.99326512e-02, -3.24822843e-01,
        1.45823106e-01, -2.13799357e-01,  6.25478402e-02, -3.12967189e-02,
       -5.41984618e-01, -1.68673798e-01, -4.48031485e-01, -7.54418969e-02,
       -3.69284064e-01, -3.00064087e-01, -3.22068691e-01,  2.18135253e-01,
       -4.65191416e-02,  2.68218249e-01, -1.33386338e+00,  5.39563417e-01,
       -1.83845520e-01, -3.96471769e-01, -7.36096382e-01,  4.21031415e-01,
        4.75515246e-01,  

In [36]:
from pinecone_text.sparse import SpladeEncoder

splade = SpladeEncoder()

tokenizer_config.json: 100%|██████████| 466/466 [00:00<00:00, 466kB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 696kB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 2.80MB/s]
config.json: 100%|██████████| 670/670 [00:00<?, ?B/s] 
pytorch_model.bin: 100%|██████████| 438M/438M [00:51<00:00, 8.46MB/s] 


In [37]:
sparse_embeds = splade.encode_queries(data[0]["context"])
sparse_embeds

{'indices': [1000,
  1039,
  1052,
  1997,
  1999,
  2003,
  2024,
  2049,
  2083,
  2094,
  2173,
  2239,
  2278,
  2290,
  2306,
  2331,
  2415,
  2427,
  2523,
  2537,
  2550,
  2565,
  2566,
  2597,
  2644,
  2754,
  2757,
  2832,
  2974,
  3030,
  3081,
  3102,
  3252,
  3269,
  3274,
  3280,
  3370,
  3392,
  3399,
  3508,
  3526,
  3571,
  3581,
  3628,
  3727,
  3740,
  3817,
  3965,
  3968,
  4264,
  4295,
  4372,
  4442,
  4456,
  4574,
  4649,
  4717,
  4730,
  4758,
  4775,
  4870,
  4962,
  4963,
  5080,
  5104,
  5258,
  5397,
  5701,
  5708,
  5920,
  5996,
  6198,
  6210,
  6215,
  6310,
  6418,
  6470,
  6531,
  6546,
  6580,
  6897,
  7053,
  7337,
  7366,
  7403,
  7473,
  7609,
  7691,
  7775,
  7816,
  8475,
  8676,
  8715,
  8761,
  8765,
  8872,
  8979,
  9007,
  9232,
  9448,
  9607,
  9706,
  9890,
  9895,
  9915,
  10012,
  10088,
  10244,
  10267,
  10327,
  10507,
  10708,
  10738,
  11503,
  11568,
  11704,
  11767,
  11798,
  11829,
  11934,
  12222,
  124

In [55]:
index = pinecone.Index("medical-qa")
index

In [57]:
from tqdm import tqdm

batch_size = 200
with tqdm(total = len(data)//batch_size +1) as pbar:
    for i in range(0, len(data), batch_size):
        i_end = min(i+batch_size, len(data))
        ids = [x["id"] for x in data[i:i_end]]
        contexts = [x["context"] for x in data[i:i_end]]
        sparse_embeds = splade.encode_queries(contexts)
        dense_embeds = dense_model.encode(contexts)
        upserts = []
        for _id, _context, _sparse, _dense in zip(ids, contexts, sparse_embeds, dense_embeds):
            upserts.append({"id": _id,
                            "values": _dense,
                            "sparse_values": _sparse,
                            "metadata": {"context":_context}})
        index.upsert(upserts)
        pbar.update(1)

index.describe_index_stats()

100%|██████████| 30/30 [13:15<00:00, 26.52s/it]


{'dimension': 768,
 'index_fullness': 0.058,
 'namespaces': {'': {'vector_count': 5800}},
 'total_vector_count': 5800}

In [62]:
vector = dense_model.encode(data[0]["context"]).tolist()
sparse_vector = splade.encode_queries(data[0]["context"])
result = index.query(
    top_k = 15,
    vector = vector,
    sparse_vector= sparse_vector,
    include_metadata=True
)
result

{'matches': [{'id': '21645374-0',
              'metadata': {'context': 'Programmed cell death (PCD) is the '
                                      'regulated death of cells within an '
                                      'organism. The lace plant (Aponogeton '
                                      'madagascariensis) produces perforations '
                                      'in its leaves through PCD. The leaves '
                                      'of the plant consist of a latticework '
                                      'of longitudinal and transverse veins '
                                      'enclosing areoles. PCD occurs in the '
                                      'cells at the center of these areoles '
                                      'and progresses outwards, stopping '
                                      'approximately five cells from the '
                                      'vasculature.'},
              'score': 311.469482,
              'values':

In [67]:
result["matches"][0]["metadata"]["context"]

'Programmed cell death (PCD) is the regulated death of cells within an organism. The lace plant (Aponogeton madagascariensis) produces perforations in its leaves through PCD. The leaves of the plant consist of a latticework of longitudinal and transverse veins enclosing areoles. PCD occurs in the cells at the center of these areoles and progresses outwards, stopping approximately five cells from the vasculature.'

In [73]:
def hybrid_scale(dense, sparse, alpha: int):
    if alpha<0 or alpha>1:
        raise ValueError("Alpha should be between 0 and 1")
    
    dense_vector = [v*alpha for v in dense]
    sparse_vector = {
        'indices': sparse['indices'],
        'values':  [v * (1 - alpha) for v in sparse['values']]
    }

    return dense_vector, sparse_vector

In [77]:
query = "Can clinicians use the PHQ-9 to assess depression in people with vision loss?"
vector = dense_model.encode(query).tolist()
sparse_vector = splade.encode_queries(query)

vector, sparse_vector = hybrid_scale(vector, sparse_vector, alpha = 0.5)
result = index.query(
    top_k = 15,
    vector = vector,
    sparse_vector= sparse_vector,
    include_metadata=True
)
result

{'matches': [{'id': '19156007-0',
              'metadata': {'context': 'To investigate whether the Patient '
                                      'Health Questionnaire-9 (PHQ-9) '
                                      'possesses the essential psychometric '
                                      'characteristics to measure depressive '
                                      'symptoms in people with visual '
                                      'impairment. The PHQ-9 scale was '
                                      'completed by 103 participants with low '
                                      'vision. These data were then assessed '
                                      'for fit to the Rasch model. The '
                                      "participants' mean +/- standard "
                                      'deviation (SD) age was 74.7 +/- 12.2 '
                                      'years.'},
              'score': 101.817055,
              'values': []},
             {'id': 

In [75]:
data[5]

{'id': '21645374-5',
 'context': 'A TUNEL assay showed fragmented nDNA in a gradient over these mitochondrial stages. Chloroplasts and transvacuolar strands were also examined using live cell imaging. The possible importance of mitochondrial permeability transition pore (PTP) formation during PCD was indirectly examined via in vivo cyclosporine A (CsA) treatment. This treatment resulted in lace plant leaves with a significantly lower number of perforations compared to controls, and that displayed mitochondrial dynamics similar to that of non-PCD cells.'}